## Data Science Position Task

#### 1-  Objective and Hypothesis

###### The objective of this task is to conclude whether or not people in the Muslim world die more often in month of Shaban. I will be using a ###### combination of hypothesis testing and data exploration to support any claims.

###### The null hypothesis (H<sub>0</sub>) is that average number of the deaths for all months are the same. 
###### The alternate hypothesis (H<sub>a</sub>) is that the average number of deaths of the month of Shaban is more than all months.

#### 2- Data Collection 

#### 3- Data Exploration and Cleaning

#### 3- Modeling 

#### 4- Results and Summary